[View in Colaboratory](https://colab.research.google.com/github/Parth-Patel12/Keras-Examples/blob/master/CIFAR_10_ALexNet.ipynb)

In [0]:
import keras
from keras.models import Sequential
from keras.datasets import cifar10
from keras.layers import Flatten,Dense,Dropout,Conv2D,MaxPooling2D,BatchNormalization
from keras.optimizers import adam

In [0]:
#Define hyper parameters

batch_size = 128
classes = 10
epochs = 100


In [0]:
#load cifar10

(x_train,y_train) , (x_test,y_test) = cifar10.load_data()

In [0]:
im_height , im_width , channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

In [0]:
#one hot encoding
y_train = keras.utils.to_categorical(y_train,classes)
y_test = keras.utils.to_categorical(y_test,classes)

In [0]:
#AlexNet
from keras import regularizers
weight_decay = 1e-4
model = Sequential()
# original model.add(Conv2D(96, (11,11), strides=(4,4), activation='relu', padding='same', input_shape=(img_height, img_width, channel,)))
model.add(Conv2D(96,(3,3),strides=(2,2),activation='relu',padding='same' ,kernel_regularizer=regularizers.l2(weight_decay),input_shape=(im_height,im_width,channel,)))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(BatchNormalization())
#additional dropout
model.add(Dropout(0.2))


model.add(Conv2D(256,(5,5),activation='relu',padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
model.add(BatchNormalization())

model.add(Dropout(0.25))


model.add(Conv2D(384, (3,3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Dropout(0.3))
model.add(Conv2D(384, (3,3), activation='relu', padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Dropout(0.35))
model.add(Conv2D(256, (3,3), activation='relu', padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(classes, activation='softmax'))




In [0]:
#summary
model.summary()

In [0]:
from keras.callbacks import LearningRateScheduler
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping

def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    elif epoch > 100:
        lrate = 0.0003        
    return lrate
  
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

#training
batch_size = 64

opt_rms = keras.optimizers.rmsprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=batch_size,
                    epochs=100,
                    verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule)])


In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [0]:
model_json = model.to_json()
with open('model.json', 'w') as json_file:
    json_file.write(model_json)

model.save_weights('model.h5')    
